In [1]:
# ! pip install yfinance
# ! pip install MetaTrader5

# https://pypi.org/project/TA-Lib/

In [2]:
from function.preprocess_function import  *
from function.vizualization import  *

from sklearn.ensemble import RandomForestRegressor

from sklearn.feature_selection import RFE

import warnings
warnings.filterwarnings("ignore")


ModuleNotFoundError: No module named 'sklearn'

In [ ]:
2/0

# data from yfinance

In [ ]:
current_datetime = datetime.now()
start_date = '2000-01-01'
end_date = current_datetime.strftime('%Y-%m-%d')
# '^GDAXI'
data = yf.download('EURUSD=X', start= start_date, end=end_date, interval='1d')
data['Volume'] = data['High'] - data['Low']
data['PositivePriceChange'] =  data['High'] -  data['Open'] 
data['NegativePriceChange'] = data['Open'] - data['Low']
# data['PriceDirection'] = np.where(data['PositivePriceChange'] > data['NegativePriceChange'], 'Up', 'Down')



data = data.reset_index()
data['Date'] = pd.to_datetime(data['Date']).dt.strftime('%Y-%m-%d' ) # %H:%M:%S')

data[:50]

In [ ]:
data.shape

In [ ]:
data.hist(figsize = (10,10));

# data preprocessing

In [ ]:
# preprocessor = Preprocessing_stock_data(data)
# all_indicator = preprocessor.all_methods()

# data = preprocessor.support_resistance_line()
# data

In [ ]:
# all_indicator.sample(3)

# experiments 



Etap I. Określanie kierunku.
1. Wyznaczam cenę max i min dla różnych przedziałów od 1 do 60 dni wstecz.
2. Określam czy cena w bieżącym dniu przekroczyła wyznaczony poziom min lub max.
3. Określam czy cena w bieżącym dniu powróciła do określonego przedziału między min a max.
4. Wyznaczam bieżący kieru5.ek:
- po przekroczeniu i powrocie do poziomu min ("wsparcie") kierunek to6.BUY;
- po przekroczeniu i powrocie do poziomu max ("opór") kierunek to7SELL;
5. Po zakończeniu sesji powtarzam punk
8. Sprawdzam w którym kierunku następnego dnia był większy ruch. W górę to cena max - otwarcie. W dół to cena otwarcie - min.ty 1-4 d

In [ ]:
# support_resistance_chart(data) 

In [ ]:
def define_level(data: pd.DataFrame, window_sizes: list = [100], bias: int = 1) -> pd.DataFrame:
    data = data.copy()  
    
    for window_size in window_sizes:  
        data.loc[:, f'RollingMax_{window_size}'] = data['High'].rolling(window=window_size).max().shift(bias)
        data.loc[:, f'RollingMin_{window_size}'] = data['Low'].rolling(window=window_size).min().shift(bias)
       
    return data

data_with_breakout_level = define_level(data, [14], 1) #  [i for i in range(14,365,1)])
data_with_breakout_level

In [ ]:
print(data_with_breakout_level.isna().sum())
data_with_breakout_level.dropna( axis = 0, inplace = True)

In [ ]:
data_with_breakout_level.reset_index(drop = True, inplace= True)
data_with_breakout_level# .reset_index(drop = True, inplace= True)


In [ ]:
# Analiza odbicia cen od poziomów oporu
data_with_breakout_level['Breakout_from_Resistance'] = ((data_with_breakout_level["RollingMax_14"] < data_with_breakout_level["High"])  & (data_with_breakout_level["RollingMax_14"] > data_with_breakout_level["Close"])).astype(int)
resistance_indices = data_with_breakout_level[data_with_breakout_level['Breakout_from_Resistance'] == True].index + 1
data_with_breakout_level.loc[resistance_indices, 'Next_Resistance_Candle_Type'] = (data_with_breakout_level.loc[resistance_indices, 'Open'] > data_with_breakout_level.loc[resistance_indices, 'Close']).astype(int)

# Analiza odbicia cen od poziomów wsparcia
data_with_breakout_level['Breakout_from_Support'] = ((data_with_breakout_level["RollingMin_14"] > data_with_breakout_level["Low"])  & (data_with_breakout_level["RollingMin_14"] < data_with_breakout_level["Close"])).astype(int)
support_indices = data_with_breakout_level[data_with_breakout_level['Breakout_from_Support'] == True].index + 1
data_with_breakout_level.loc[support_indices, 'Next_Support_Candle_Type'] = (data_with_breakout_level.loc[support_indices, 'Open'] < data_with_breakout_level.loc[support_indices, 'Close']).astype(int)

# Вывод результатов анализа
data_with_breakout_level['Breakout_from_Resistance'].sum(), data_with_breakout_level['Breakout_from_Support'].sum()
# data_with_breakout_level



# # Wyłączenie poprzednich kolumn i dodanie nowych kolumn do analizy przebicia poziomów oporu i wsparcia
# data_with_breakout_level['Przerwa_w_Wsparciu'] = data_with_breakout_level["RollingMin_14"] > data_with_breakout_level["Close"]
# data_with_breakout_level['Przerwa_w_Oporze'] = data_with_breakout_level["RollingMax_14"] < data_with_breakout_level["Close"]

# # Analiza sumy przebiciu poziomów wsparcia i oporu
# data_with_breakout_level['Przerwa_w_Wsparciu'].sum(), data_with_breakout_level['Przerwa_w_Oporze'].sum()

# # Wyświetlenie wierszy, gdzie warunek przebicia poziomu oporu jest spełniony
# data_with_breakout_level[data_with_breakout_level['Przebicie_oporu'] == True]


[RFE](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html)

In [ ]:
def calculate_score(data):
    # Расчет средней прибыльности
    returns = data['Close'].pct_change()  # Вычисление процентного изменения цен закрытия
    average_return = returns.mean()  # Вычисление среднего значения процентного изменения
    return average_return

def optimize_parameters(data, parameter_ranges):
    best_score = 0
    best_params = None

    for window_size in parameter_ranges['window_size']:
        for bias in parameter_ranges['bias']:
            current_data = define_level(data, [window_size], bias)  # Передаем window_size как список
            # Здесь вы можете реализовать другие методы оптимизации, такие как расчет метрики эффективности и сравнение с лучшим результатом
            current_score = calculate_score(current_data)  # Функция для расчета метрики эффективности

            if current_score > best_score:
                best_score = current_score
                best_params = {'window_size': window_size, 'bias': bias}

    return best_params

# Пример использования оптимизации параметров
parameter_ranges = {'window_size': [i for i in range(20, 365, 1)], 
                    'bias': [i for i in range(1, 10, 1)]}

best_parameters = optimize_parameters(data, parameter_ranges)
print(f"Лучшие параметры: {best_parameters}")

# AI

In [ ]:
def create_dataset(dataset, time_step=25):
    dataX, dataY = [], []
    dataset_without_close = dataset.drop('Close', axis=1)

    for i in range(len(dataset) - time_step - 1):
        if (i + time_step) < len(dataset):
            a = dataset_without_close.iloc[i:(i + time_step), :]
            dataX.append(a.values)
            dataY.append(dataset.iloc[i + time_step]['Close'])  # Добавлено изменение в индексации здесь
    return np.array(dataX), np.array(dataY)

In [ ]:
def create_dataset(dataset, time_step):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)